Credit for this super helpful Colab notebook (and package) goes to [Max Woolf](http://minimaxir.com). Retrain the GPT-2 net on your choice of text by making a copy of the notebook [here](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce)!

In [ ]:
%tensorflow_version 1.x
#!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 233Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 90.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 369Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 171Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 262Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 177Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 193Mit/s]                                                       


In [ ]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AGr5ZFPskAWvwhOLfxJwCwhxjolSk0FOC7iG68foxIy8SgiIVSet-c
Mounted at /content/drive


In [ ]:
file_name = "corpus-gpt2.txt"

## Finetune GPT-2

Helpful parameters:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500)

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

## Load a Trained Model Checkpoint


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model


Helpful parameters:

* **`prefix`**: Force the text to start with a given character sequence and generate text from there
* **`nsamples`**: Generate multiple texts at a time
* **`batch_size`**: Generate multiple samples in parallel (unique to GPT-2!)
*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.
* **`return_as_list`**: Boolean, returns string samples in a list if True

In [ ]:
text = gpt2.generate(sess,
              #top_k=10,
              top_p=0.9,
              length=100,
              temperature=0.8,
              prefix=' ',
              nsamples=1,
              #batch_size=5,
              return_as_list=True)
text[0]

'   \n\nTowards the end of the night, I decided to go for a run. I had dropped 5 hits on myself and my friend at my house, but I had no idea how good they were. My friend had spent the weekend in a mental institution and I had just dropped 3 of my last 8 hits. It was like nothing had happened that day. I had the drug experience in my hands. \n\nThe next day I felt a bit light headed and a bit'

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.